## title

In [1]:
(define (constant value connector)
  (define (me request)
    (error "Unknow request -- CONSTANT" request))
  (connect connector me)
  (set-value! connector value me)
  me)

(define (probe name connector)
  (define (print-probe value)
    (newline)
    (display "Probe: ")
    (display name)
    (display " = ")
    (display value))
  (define (process-new-value)
    (print-probe (get-value connector)))
  (define (process-forget-value)
    (print-probe "?"))
  (define (me request)
    (cond ((eq? request 'I-have-a-value)
	   (process-new-value))
	  ((eq? request 'I-lost-my-value)
	   (process-forget-value))
	   (else
	    (error "Unknow request -- PROBE " request))))
    (connect connector me)
    me)

(define (make-connector)
  (let ((value false) (informant false) (constraints '()))
    (define (set-my-value newval setter)
      (cond ((not (has-value? me))
	     (set! value newval)
	     (set! informant setter)
	     (for-each-except setter
			      inform-about-value
			      constraints))
	    ((not (= value newval))
	     (error "Constradiction" (list value newval)))
	    (else 'ignored)))
    (define (forget-my-value retractor)
      (if (eq? retractor informant)
	  (begin (set! informant false)
		 (for-each-except retractor
				  inform-about-no-value
				  constraints))
	  'ignored))
    (define (connect new-constraint)
      (if (not (memq new-constraint constraints))
	  (set! constraints 
		(cons new-constraint constraints)))
      (if (has-value? me)
	  (inform-about-value new-constraint))
      'done)
    (define (me request)
      (cond ((eq? request 'has-value?)
	     (if informant true false))
	    ((eq? request 'value ) value)
	    ((eq? request 'set-value!) set-my-value)
	    ((eq? request 'forget) forget-my-value)
	    ((eq? request 'connect) connect)
	    (else (error "Unknown operation -- CONNECTOR"
			 request))))
    me))

(define (inform-about-value constraint)
  (constraint 'I-have-a-value))

(define (inform-about-no-value constraint)
  (constraint 'I-lost-my-value))

(define (for-each-except exception procedure list)
  (define (loop items)
    (cond ((null? items) 'done)
	  ((eq? (car items) exception ) (loop ( cdr items)))
	  (else (procedure (car items))
		(loop (cdr items)))))
  (loop list))

(define (has-value? connector)
  (connector 'has-value?))
(define (get-value connector)
  (connector 'value))
(define (set-value! connector new-value informant)
  ((connector 'set-value!) new-value informant))
(define (forget-value! connector retractor)
  ((connector 'forget) retractor))
(define (connect connector new-constraint)
  ((connector 'connect) new-constraint))

(define (adder a1 a2 sum)
  (define (process-new-value)
    (cond ((and (has-value? a1) (has-value? a2))
	   (set-value! sum
		       (+ (get-value a1) (get-value a2))
		       me))
	  ((and (has-value? a1) (has-value? sum))
	   (set-value! a2
		       (- (get-value sum) (get-value a1))
		       me))
	  ((and (has-value? a2) (has-value? sum))
	   (set-value! a1
		       (- (get-value sum) (get-value a2))
		       me))))
  (define (process-forget-value)
    (forget-value! sum me)
    (forget-value! a1 me)
    (forget-value! a2 me)
    (process-new-value))
  (define (me request)
    (cond ((eq? request 'I-have-a-value)
	   (process-new-value))
	  ((eq? request 'I-lost-my-value)
	   (process-forget-value))
	  (else
	   (error "Unknown request -- ADDER" request))))
  (connect a1 me)
  (connect a2 me)
  (connect sum me)
  me)

(define (multiplier m1 m2 product)
  (define (process-new-value)
    (cond ((or (and (has-value? m1) (= (get-value m1) 0))
	       (and (has-value? m2) (= (get-value m2) 0)))
	   (set-value! product 0 me))
	  ((and (has-value? m1) (has-value? m2))
	   (set-value! product
		       (* (get-value m1) (get-value m2))
		       me))
	  ((and (has-value? product) (has-value? m1))
	   (set-value! m2
		       (/ (get-value product) (get-value m1))
		       me))
	  ((and (has-value? product) (has-value? m2))
	   (set-value! m1
		       (/ (get-value product) (get-value m2))
		       me))))
  (define (process-forget-value)
    (forget-value! product me)
    (forget-value! m1 me)
    (forget-value! m2 me)
    (process-new-value))
  (define (me request)
    (cond ((eq? request 'I-have-a-value)
	   (process-new-value))
	  ((eq? request 'I-lost-my-value)
	   (process-forget-value))
	  (else
	   (error "Unknown request -- MULTIPLIER " request))))
  (connect m1 me)
  (connect m2 me)
  (connect product me)
  me)

(define (start-unit-test-adder)
  (define value1 (make-connector))
  (define value2 (make-connector))
  (define my-sum (make-connector))

  (adder value1 value2 my-sum)

  (probe 'value1 value1)
  (probe 'value2 value2)
  (probe 'my-sum my-sum)

  (set-value! value1 1 'user)
  (set-value! value2 2 'user)

  (forget-value! value1 'user)
;  (forget-value! value2 'user)

  (set-value! value1 4 'user)

  (forget-value! value1 'user)
  (forget-value! my-sum 'user)
  
  (set-value! my-sum 19 'user)
 
 )


(define (start-unit-test-multiplier)
  (define value1 (make-connector))
  (define value2 (make-connector))
  (define my-product (make-connector))

  (multiplier value1 value2 my-product)

  (probe 'value1 value1)
  (probe 'value2 value2)
  (probe 'my-product my-product)

  (set-value! value1 1 'user)
  (set-value! value2 2 'user)

  (forget-value! value1 'user)
;  (forget-value! value2 'user)

  (set-value! value1 4 'user)

  (forget-value! value1 'user)
  (forget-value! my-product 'user)
  
  (set-value! my-product 19 'user)
 
 )

(define (averager a b c)
  (define number-2  (make-connector))
  (define sum-value (make-connector))
  (adder a b sum-value)

  (multiplier c number-2 sum-value)
  (constant 2 number-2)
  'ok)

(define (start-test-3-33)
  (define a (make-connector))
  (define b (make-connector))
  (define c (make-connector))
  
  (averager a b c)

  (probe 'a a)
  (probe 'b b)
  (probe 'c c)

  (set-value! a 3 'user)
  (set-value! b 5 'user)

)
    






















